In [1]:
import requests, csv, time
from collections import deque
from urllib.parse import urljoin, urlparse, urlunparse
from urllib.robotparser import RobotFileParser
from lxml.html import fromstring
from lxml.etree import ParserError

class WebCrawler:
    """Crawler simplificado para rastrear y exportar enlaces del sitio UNTREF"""
    def __init__(self, url_base, archivo_salida="enlaces.csv", demora_segundos=2):
        self.url_base = self.normalizar_url(url_base)
        self.archivo_salida = archivo_salida
        self.visitados = set()
        self.enlaces = set()
        self.demora_segundos = demora_segundos
        self.robot_parser = self.cargar_robots_txt()
        self.total_paginas_visitadas = 0

    def normalizar_url(self, url):
        """Convierte la URL a su versión con https y elimina cualquier fragmento o query."""
        parsed_url = urlparse(url)
        return urlunparse(parsed_url._replace(scheme="https", fragment="", query=""))

    def cargar_robots_txt(self):
        rp = RobotFileParser()
        robots_url = urljoin(self.url_base, "/robots.txt")
        try:
            rp.set_url(robots_url)
            rp.read()
        except Exception as e:
            print(f"Error al leer {robots_url}: {e}")
        return rp

    def es_rastreable(self, url):
        return self.robot_parser.can_fetch("*", url)
    
    def es_dominio_edu_ar(self, url):
        return urlparse(url).netloc.startswith("untref.edu.ar")

    def es_url_valida(self, url):
        """Verifica si la URL es válida y no contiene rutas no deseadas."""
        extensiones_invalidas = (".pdf", ".jpg", ".jpeg", ".png", ".gif", ".doc", ".docx", ".xls", ".xlsx", ".zip", ".rar", ".mp3", ".mp4", ".exe")
        if any(url.lower().endswith(ext) for ext in extensiones_invalidas):
            return False
        if "/download/" in url.lower() or "/downloads/" in url.lower() or "/ceipsu/" in url.lower():
            return False
        return True

    def crawl(self, url=None):
        if url is None:
            url = self.url_base

        queue = deque([url])

        while queue:
            current_url = queue.popleft()
            current_url = self.normalizar_url(current_url)  # Normalizar la URL
            if current_url in self.visitados:
                continue
            if not self.es_rastreable(current_url):
                continue

            self.visitados.add(current_url)
            self.total_paginas_visitadas += 1
            print(f"Visitando: {current_url} | Total páginas visitadas: {self.total_paginas_visitadas}")

            try:
                respuesta = requests.get(current_url)
                if respuesta.status_code != 200:
                    continue
            except requests.RequestException:
                continue

            if not respuesta.content.strip():
                continue

            try:
                html = fromstring(respuesta.content)
            except ParserError:
                continue

            for enlace in html.xpath("//a[@href]"):
                url_destino = urljoin(self.url_base, enlace.get("href"))
                url_destino = self.normalizar_url(url_destino)  # Normalizar la URL destino
                if (self.es_dominio_edu_ar(url_destino) and 
                    self.es_url_valida(url_destino) and 
                    url_destino not in self.visitados):
                    self.enlaces.add((current_url, url_destino))
                    queue.append(url_destino)

            time.sleep(self.demora_segundos)

    def guardar_en_csv(self):
        with open(self.archivo_salida, "w", newline='', encoding="utf-8") as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(["URL Origen", "URL Destino"])
            for enlace in self.enlaces:
                escritor.writerow(enlace)

    def iniciar(self):
        print(f"Iniciando el crawler en: {self.url_base}")
        self.crawl(self.url_base)
        self.guardar_en_csv()
        print(f"Crawler finalizado. Total de páginas visitadas: {self.total_paginas_visitadas}")
        print(f"Total de enlaces guardados: {len(self.enlaces)}")


In [2]:
# Ejemplo de uso
crawler = WebCrawler("https://untref.edu.ar")
crawler.iniciar()

Iniciando el crawler en: https://untref.edu.ar
Visitando: https://untref.edu.ar | Total páginas visitadas: 1
Visitando: https://untref.edu.ar/autoridades | Total páginas visitadas: 2
Visitando: https://untref.edu.ar/sedes | Total páginas visitadas: 3
Visitando: https://untref.edu.ar/docentes_departamento | Total páginas visitadas: 4
Visitando: https://untref.edu.ar/untref-en-cifras | Total páginas visitadas: 5
Visitando: https://untref.edu.ar/llamado-a-concurso-cerrado-de-oposicion-y-antecedentes-para-cubrir-cargos-del-personal-no-docente | Total páginas visitadas: 6
Visitando: https://untref.edu.ar/autoevaluacion-institucional | Total páginas visitadas: 7
Visitando: https://untref.edu.ar/normativa-institucional | Total páginas visitadas: 8
Visitando: https://untref.edu.ar/comunicacion-y-prensa | Total páginas visitadas: 9
Visitando: https://untref.edu.ar/elecciones-untref-2024 | Total páginas visitadas: 10
Visitando: https://untref.edu.ar/comite-interdisciplinario-de-violencia-de-g%C3